# Power Outage

**Name(s)**: Luke, Andrew

**Website Link**: (your website link)

## Code

In [3]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

### Cleaning and EDA

In [5]:
outage = pd.read_excel("outage.xlsx", sheet_name="Masterdata")
outage.head().to_clipboard()

PermissionError: [Errno 13] Permission denied: 'outage.xlsx'

In [29]:
# Drop Info Rows 
df = outage.drop(range(4)).dropna(axis=1, how='all')

# Set column names
df.columns = df.iloc[0]

# Drop Unit row and variables
df = df.drop([4, 5])
df = df.drop(columns="variables")

# Combine 'OUTAGE.START.DATE' and 'OUTAGE.START.TIME'
df['OUTAGE.START'] = pd.to_datetime(df['OUTAGE.START.DATE']) + pd.to_timedelta(df['OUTAGE.START.TIME'].astype(str))

# Combine 'OUTAGE.RESTORATION.DATE' and 'OUTAGE.RESTORATION.TIME'
df['OUTAGE.RESTORATION'] = pd.to_datetime(df['OUTAGE.RESTORATION.DATE']) + pd.to_timedelta(df['OUTAGE.RESTORATION.TIME'].astype(str))

# Drop the original date and time columns
df = df.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], axis=1)

# Fill Missing 
df.replace("NA", np.nan, inplace=True)

# Display 
df[['RES.SALES']]

df.to_csv("outage.csv")

In [20]:
len(df["variables"][df["variables"].isna()])

1534

### Assessment of Missingness

In [ ]:
# TODO

### Hypothesis Testing

In [ ]:
# TODO